In [3]:
from rsna_retro.imports import *
from rsna_retro.metadata import *
from rsna_retro.preprocess import *
from rsna_retro.train import *

In [4]:
torch.cuda.set_device(3)

In [5]:
df_comb.head()

,Modality,PatientID,StudyInstanceUID,SeriesInstanceUID,StudyID,ImagePositionPatient,ImageOrientationPatient,SamplesPerPixel,PhotometricInterpretation,Rows,...,MultiWindowCenter,WindowCenter1,MultiWindowWidth,WindowWidth1,any,epidural,intraparenchymal,intraventricular,subarachnoid,subdural
SOPInstanceUID,,,,,,,,,,,,,,,,,,,,,
ID_352e89f1c,CT,ID_d557ddd2,ID_05074a0d95,ID_be6165332c,,-125.000000,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_3cf4fb50f,CT,ID_16b2ad86,ID_c3a404ea2e,ID_2c1454e208,,-125.000000,1.0,1,MONOCHROME2,512,...,1.0,36.0,1.0,80.0,0,0,0,0,0,0
ID_e3674b189,CT,ID_eb712bf0,ID_db83193795,ID_e1facea145,,-125.000000,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0
ID_2a8702d25,CT,ID_ff137633,ID_d17053848c,ID_7098f7c836,,-126.437378,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,1,0,1,1,0,0
ID_7be0f1b3c,CT,ID_cd9169c2,ID_b42de79024,ID_f5bd86b25b,,-125.000000,1.0,1,MONOCHROME2,512,...,NaN,NaN,NaN,NaN,0,0,0,0,0,0


In [11]:
fns = L(list(df_comb.index))

In [12]:
def get_data(bs, sz, img_dir='nocrop_jpg', splits=splits):
    if sz <= 256: img_dir = f'{img_dir}256'
    return get_data_gen(fns, bs=bs, img_tfm=get_pil_fn(path/img_dir), 
                        sz=sz, splits=splits)

## Training

In [17]:
name = 'baseline'

In [18]:
dbch = get_data(512, 128)
learn = get_learner(dbch, xresnet34, get_loss(), pretrained=False)

## Training

In [18]:
dbch = get_data(512, 128)
learn = get_learner(dbch, xresnet34, get_loss(), pretrained=False)
do_fit(learn, 20, 4e-2, freeze=False)
learn.save(f'runs/{name}-1')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.125112,0.216901,0.922390,0.871643,10:26
1,0.111086,0.113133,0.962970,0.935694,07:32
2,0.103534,0.110283,0.964507,0.939262,07:38
3,0.102804,0.126087,0.958786,0.918361,07:34
4,0.099555,0.111889,0.966204,0.933888,07:35
5,0.097559,0.106072,0.966636,0.933230,07:36
6,0.093827,0.100732,0.969573,0.941238,07:35
7,0.093589,0.114386,0.965277,0.928143,07:36
8,0.090966,0.096022,0.970488,0.941588,07:38
9,0.089289,0.101036,0.969437,0.940176,07:35


In [19]:
learn.dbunch = get_data(512, 224)
do_fit(learn, 12, 5e-3, freeze=False)
learn.save(f'runs/{name}-2')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.070574,0.080748,0.974566,0.952262,15:53
1,0.071542,0.077432,0.974982,0.953430,15:42
2,0.069703,0.081170,0.974146,0.953016,15:41
3,0.069116,0.076168,0.975313,0.953663,15:40
4,0.067661,0.078817,0.975290,0.952729,15:42
5,0.065829,0.084029,0.974840,0.951157,15:43
6,0.062878,0.077341,0.975922,0.954184,15:41
7,0.059482,0.078382,0.975547,0.953791,15:40
8,0.056008,0.077781,0.975879,0.954460,15:41
9,0.051525,0.078849,0.975922,0.954906,15:44


In [20]:
learn.dbunch = get_data(256, 384)
do_fit(learn, 4, 1e-3, freeze=False)
learn.save(f'runs/{name}-3')

epoch,train_loss,valid_loss,accuracy_multi,accuracy_any,time
0,0.067043,0.076415,0.975357,0.953547,47:00
1,0.060787,0.074738,0.976364,0.955671,45:30
2,0.055624,0.075182,0.976244,0.955129,45:24
3,0.049732,0.077263,0.976012,0.955108,45:27


## Submission

In [19]:
learn.load(f'runs/{name}-3')

In [20]:
# tst_fns = df_tst.index.values

In [6]:
sub_fn = f'subm/{name}'

In [ ]:
learn.dbunch = get_test_data(df_tst, bs=256, sz=384)

In [21]:
# tst_bs = 256
# tst_sz = 384
# tst_splits = [L.range(tst_fns), L.range(tst_fns)]
# tst_dbch = get_data_gen(tst_fns, bs=tst_bs, img_tfm=get_pil_fn(path/'tst_jpg'), sz=tst_sz, splits=tst_splits, test=True)
# learn.dbunch = tst_dbch

In [23]:
preds,targs = learn.get_preds()

In [33]:
pred_csv = submission(df_tst, preds, fn=sub_fn)

In [34]:
FileLink(f'{nm}.csv')

/home/ashaw/kaggle/rsna_retro/subm/test_sub.csv

In [7]:
api.competition_submit(f'{nm}.csv', 'testing rsna_retro submission', 'rsna-intracranial-hemorrhage-detection')

100%|██████████| 26.0M/26.0M [00:03<00:00, 8.99MB/s]


Successfully submitted to RSNA Intracranial Hemorrhage Detection

In [10]:
last_subm_id = api.competition_submissions('rsna-intracranial-hemorrhage-detection')[0]

In [15]:
api.competitions_submissions_list('rsna-intracranial-hemorrhage-detection')[0]

{'ref': 13695014,
 'totalBytes': 27277209,
 'date': '2019-12-08T22:49:07.013Z',
 'description': 'testing rsna_retro submission',
 'errorDescription': None,
 'fileName': 'test_sub.csv',
 'publicScore': '0.95475',
 'privateScore': '0.06444',
 'status': 'complete',
 'submittedBy': 'Andrew Shaw',
 'submittedByRef': 'bearpelican',
 'teamName': 'Andrew Shaw',
 'type': 'standard',
 'url': 'https://www.kaggle.com/submissions/13695014/13695014.raw'}